In [1]:
import json
import numpy as np
import pandas as pd
import requests

In [242]:
dataset = pd.read_csv('test-data/weather-10-cities.csv')

In [243]:
# A test dataset with 10 cities to test the process with
dataset.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,Seattle,Seattle,WA,Washington,53033,King,53033,King,47.6211,-122.3244,3643765,3336,polygon,False,True,America/Los_Angeles,1,98109 98108 98104 98107 98106 98101 98103 9810...,1840021117
1,Kansas City,Kansas City,MO,Missouri,29095,Jackson,29095|29047|29165|29037,Jackson|Clay|Platte|Cass,39.1239,-94.5541,1615621,599,polygon,False,True,America/Chicago,1,64161 64163 64165 64167 64053 64151 64153 6415...,1840008535
2,New York,New York,NY,New York,36061,New York,36061,New York,40.6943,-73.9249,19354922,11083,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
3,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,6037,Los Angeles,34.1139,-118.4068,12815475,3295,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
4,Denver,Denver,CO,Colorado,8031,Denver,8031,Denver,39.7621,-104.8759,2787266,1774,polygon,False,True,America/Denver,1,80264 80230 80231 80236 80237 80235 80238 8023...,1840018789


In [244]:
# Making a bounding box to search for available weather stations in the area of a city
lat = 39.1239
lng = -94.5541

# Using one half a degree in each direction from each city (~35 miles)
N_coord = lat + .5
S_coord = lat - .5
W_coord = lng - .5
E_coord = lng + .5

# Note that for this particular API the coordinates MUST be in this order: SWNE
boundingbox = 'extent=' + str(S_coord) + ',' + str(W_coord) + ',' + str(N_coord) + ',' + str(E_coord)

print(boundingbox)

extent=38.6239,-95.0541,39.6239,-94.0541


In [252]:
# Search stations within the 'boundingbox' defined above,
# only for stations that record TEMP and SUN,
# data must exist later than December 2018 for 'Global Summary of the Month', 
# limited to 500 stations (shouldn't be a constraint, basically just saying show all)

get_stations = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?' + boundingbox + '&datacategoryid=TEMP&startdate=2018-12-01&datasetid=GSOM&limit=500',
                   headers={'token':'HVVZlvqRvqKXInZGUELMPssOYPUmRtgk'})
station_json = get_stations.json()

station_json
# data = x['results']

[{'elevation': 259.1,
  'mindate': '1938-01-01',
  'maxdate': '2019-12-01',
  'latitude': 39.06361,
  'name': 'BONNER SPRINGS, KS US',
  'datacoverage': 0.9827,
  'id': 'GHCND:USC00140957',
  'elevationUnit': 'METERS',
  'longitude': -94.896},
 {'elevation': 307.8,
  'mindate': '1985-02-01',
  'maxdate': '2019-11-01',
  'latitude': 38.6645,
  'name': 'HILLSDALE LAKE, KS US',
  'datacoverage': 0.8422,
  'id': 'GHCND:USC00143686',
  'elevationUnit': 'METERS',
  'longitude': -94.8901},
 {'elevation': 265.5,
  'mindate': '2011-11-01',
  'maxdate': '2019-12-01',
  'latitude': 39.2849,
  'name': 'LEAVENWORTH 3 SW, KS US',
  'datacoverage': 0.9489,
  'id': 'GHCND:USC00144589',
  'elevationUnit': 'METERS',
  'longitude': -94.95},
 {'elevation': 243.8,
  'mindate': '2009-10-01',
  'maxdate': '2019-04-01',
  'latitude': 39.2871,
  'name': 'EXCELSIOR SPRINGS 4 S, MO US',
  'datacoverage': 0.896,
  'id': 'GHCND:USC00232700',
  'elevationUnit': 'METERS',
  'longitude': -94.2226},
 {'elevation': 300

In [379]:
station_data = station_json['results']

In [380]:
df_station_data = pd.DataFrame(station_data)
df_station_data = df_station_data.sort_values(by='datacoverage', ascending=0)
search_station_0 = df_station_data['id'].iloc[0]
search_station_1 = df_station_data['id'].iloc[1]


In [395]:
# start by checking 
get = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GSOM&datatypeid=TAVG,TMAX,TMIN,PRCP,SNOW,PSUN&stationid=' + search_station_0 + '&startdate=2016-05-01&enddate=2018-05-01&units=standard&limit=500',
                   headers={'token':'HVVZlvqRvqKXInZGUELMPssOYPUmRtgk'})
weather_json = get.json()
# weather_json
weather_data = weather_json['results']

In [408]:
weather_df = pd.DataFrame(weather_data)

weather_df

,attributes,datatype,date,station,value
0,",,,W",PRCP,2016-05-01T00:00:00,GHCND:USW00003947,9.51
1,",T,,W",SNOW,2016-05-01T00:00:00,GHCND:USW00003947,0.00
2,",W",TAVG,2016-05-01T00:00:00,GHCND:USW00003947,62.70
3,",,,W",TMAX,2016-05-01T00:00:00,GHCND:USW00003947,72.90
4,",,,W",TMIN,2016-05-01T00:00:00,GHCND:USW00003947,52.60
5,",,,W",PRCP,2016-06-01T00:00:00,GHCND:USW00003947,0.98
6,",,,W",SNOW,2016-06-01T00:00:00,GHCND:USW00003947,0.00
7,",W",TAVG,2016-06-01T00:00:00,GHCND:USW00003947,78.60
8,",,,W",TMAX,2016-06-01T00:00:00,GHCND:USW00003947,89.70
9,",,,W",TMIN,2016-06-01T00:00:00,GHCND:USW00003947,67.40


In [397]:
df_test = weather_df.pivot(index='date', columns='datatype', values='value')

In [398]:
df_test.head()

datatype,PRCP,SNOW,TAVG,TMAX,TMIN
date,,,,,
2016-05-01T00:00:00,9.51,0.0,62.7,72.9,52.6
2016-06-01T00:00:00,0.98,0.0,78.6,89.7,67.4
2016-07-01T00:00:00,8.76,0.0,78.6,87.6,69.5
2016-08-01T00:00:00,9.16,0.0,77.0,86.6,67.4
2016-09-01T00:00:00,4.94,0.0,71.4,81.0,61.8


In [399]:
import datetime as dt

In [400]:
df_test['date'] = pd.to_datetime(df_test.index, infer_datetime_format=True)
df_test['Month'] = df_test['date'].dt.month.astype(str)

df_test['PRCP-AVG'] = df_test.groupby(['Month'])['PRCP'].transform('mean')
df_test['SNOW-AVG'] = df_test.groupby(['Month'])['SNOW'].transform('mean')
df_test['TAVG-AVG'] = df_test.groupby(['Month'])['TAVG'].transform('mean')
df_test['TMAX-AVG'] = df_test.groupby(['Month'])['TMAX'].transform('mean')
df_test['TMIN-AVG'] = df_test.groupby(['Month'])['TMIN'].transform('mean')

df_test.head()

datatype,PRCP,SNOW,TAVG,TMAX,TMIN,date,Month,PRCP-AVG,SNOW-AVG,TAVG-AVG,TMAX-AVG,TMIN-AVG
date,,,,,,,,,,,,
2016-05-01T00:00:00,9.51,0.0,62.7,72.9,52.6,2016-05-01,5,6.786667,0.0,66.60,77.333333,55.90
2016-06-01T00:00:00,0.98,0.0,78.6,89.7,67.4,2016-06-01,6,3.705000,0.0,76.60,87.450000,65.75
2016-07-01T00:00:00,8.76,0.0,78.6,87.6,69.5,2016-07-01,7,6.880000,0.0,78.85,88.150000,69.50
2016-08-01T00:00:00,9.16,0.0,77.0,86.6,67.4,2016-08-01,8,9.675000,0.0,74.40,84.100000,64.70
2016-09-01T00:00:00,4.94,0.0,71.4,81.0,61.8,2016-09-01,9,3.870000,0.0,71.25,81.300000,61.20


In [401]:
df_output = df_test[['Month', 'PRCP-AVG', 'SNOW-AVG', 'TAVG-AVG', 'TMAX-AVG', 'TMIN-AVG']].copy()

In [402]:
df_output = df_output.reset_index()

In [403]:
df_output = df_output.drop('date', axis=1)

In [404]:
df_output = df_output.drop_duplicates()

In [405]:
df_output['Month'] = df_output['Month'].astype(int)

In [406]:
df_output = df_output.sort_values(by='Month', ascending=1)
df_output = df_output.set_index('Month')
df_output

datatype,PRCP-AVG,SNOW-AVG,TAVG-AVG,TMAX-AVG,TMIN-AVG
Month,,,,,
1,1.260000,2.45,30.45,39.950000,20.90
2,0.690000,0.45,38.05,49.600000,26.40
3,2.235000,0.20,45.85,55.800000,35.95
4,3.835000,0.95,52.20,62.950000,41.50
5,6.786667,0.00,66.60,77.333333,55.90
6,3.705000,0.00,76.60,87.450000,65.75
7,6.880000,0.00,78.85,88.150000,69.50
8,9.675000,0.00,74.40,84.100000,64.70
9,3.870000,0.00,71.25,81.300000,61.20


In [409]:
x = df_output.to_dict()

pd.DataFrame.from_dict(x)

# x['TMIN-AVG']

,PRCP-AVG,SNOW-AVG,TAVG-AVG,TMAX-AVG,TMIN-AVG
1,1.260000,2.45,30.45,39.950000,20.90
2,0.690000,0.45,38.05,49.600000,26.40
3,2.235000,0.20,45.85,55.800000,35.95
4,3.835000,0.95,52.20,62.950000,41.50
5,6.786667,0.00,66.60,77.333333,55.90
6,3.705000,0.00,76.60,87.450000,65.75
7,6.880000,0.00,78.85,88.150000,69.50
8,9.675000,0.00,74.40,84.100000,64.70
9,3.870000,0.00,71.25,81.300000,61.20
10,3.730000,0.00,59.75,70.300000,49.20


### Reference Materials

In [245]:
# A list of available ids that can be used as 'datacategories' 
ids_get = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datacategories?limit=500',
                   headers={'token':'HVVZlvqRvqKXInZGUELMPssOYPUmRtgk'})
ids_json = ids_get.json()

ids_json

{'metadata': {'resultset': {'offset': 1, 'count': 42, 'limit': 500}},
 'results': [{'name': 'Annual Agricultural', 'id': 'ANNAGR'},
  {'name': 'Annual Degree Days', 'id': 'ANNDD'},
  {'name': 'Annual Precipitation', 'id': 'ANNPRCP'},
  {'name': 'Annual Temperature', 'id': 'ANNTEMP'},
  {'name': 'Autumn Agricultural', 'id': 'AUAGR'},
  {'name': 'Autumn Degree Days', 'id': 'AUDD'},
  {'name': 'Autumn Precipitation', 'id': 'AUPRCP'},
  {'name': 'Autumn Temperature', 'id': 'AUTEMP'},
  {'name': 'Computed', 'id': 'COMP'},
  {'name': 'Computed Agricultural', 'id': 'COMPAGR'},
  {'name': 'Degree Days', 'id': 'DD'},
  {'name': 'Dual-Pol Moments', 'id': 'DUALPOLMOMENT'},
  {'name': 'Echo Tops', 'id': 'ECHOTOP'},
  {'name': 'Evaporation', 'id': 'EVAP'},
  {'name': 'Hydrometeor Type', 'id': 'HYDROMETEOR'},
  {'name': 'Land', 'id': 'LAND'},
  {'name': 'Miscellany', 'id': 'MISC'},
  {'name': 'Other', 'id': 'OTHER'},
  {'name': 'Overlay', 'id': 'OVERLAY'},
  {'name': 'Precipitation', 'id': 'PRCP'},


In [248]:
# A list of available ids that can be used as 'datacategories' 
ids2_get = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets?limit=500',
                   headers={'token':'HVVZlvqRvqKXInZGUELMPssOYPUmRtgk'})
ids2_json = ids2_get.json()

# Only showing GSOM, because that's what we're using for the project
ids2_json['results'][1]

{'uid': 'gov.noaa.ncdc:C00946',
 'mindate': '1763-01-01',
 'maxdate': '2019-12-01',
 'name': 'Global Summary of the Month',
 'datacoverage': 1,
 'id': 'GSOM'}